In [10]:
import os
import sys
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from scipy.io import loadmat


In [11]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [12]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [13]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [14]:
data=loadmat('data\cardio.mat')

In [15]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [17]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','ABOD','CBLOF','FB','HBOS','Iforest','KNN','LOF',"MCD","OCSVM","PCA"]

In [18]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, ABOD, CBLOF, FB, HBOS, Iforest, KNN, LOF, MCD, OCSVM, PCA]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, ABOD, CBLOF, FB, HBOS, Iforest, KNN, LOF, MCD, OCSVM, PCA]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, ABOD, CBLOF, FB, HBOS, Iforest, KNN, LOF, MCD, OCSVM, PCA]
Index: []


In [19]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
   print("\n... Processing", mat_file, '...')
   mat = loadmat(os.path.join('data', mat_file))

   X = mat['X']
   y = mat['y'].ravel()
   outliers_fraction = np.count_nonzero(y) / len(y)
   outliers_percentage = round(outliers_fraction * 100, ndigits=4)

   # construct containers for saving results
   roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

   # 60% data for training and 40% for testing
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                       random_state=random_state)

   # standardizing data for processing
   X_train_norm, X_test_norm = standardizer(X_train, X_test)

   classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
       contamination=outliers_fraction),
       'Cluster-based Local Outlier Factor': CBLOF(
           contamination=outliers_fraction, check_estimator=False,
           random_state=random_state),
       'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                         random_state=random_state),
       'Histogram-base Outlier Detection (HBOS)': HBOS(
           contamination=outliers_fraction),
       'Isolation Forest': IForest(contamination=outliers_fraction,
                                   random_state=random_state),
       'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
       'Local Outlier Factor (LOF)': LOF(
           contamination=outliers_fraction),
       'Minimum Covariance Determinant (MCD)': MCD(
           contamination=outliers_fraction, random_state=random_state),
       'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
       'Principal Component Analysis (PCA)': PCA(
           contamination=outliers_fraction, random_state=random_state),
   }

   for clf_name, clf in classifiers.items():
       t0 = time()
       clf.fit(X_train_norm)
       test_scores = clf.decision_function(X_test_norm)
       t1 = time()
       duration = round(t1 - t0, ndigits=4)
       time_list.append(duration)

       roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
       prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

       print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
             'execution time: {duration}s'.format(
           clf_name=clf_name, roc=roc, prn=prn, duration=duration))

       roc_list.append(roc)
       prn_list.append(prn)

   temp_df = pd.DataFrame(time_list).transpose()
   temp_df.columns = df_columns
   time_df = pd.concat([time_df, temp_df], axis=0)

   temp_df = pd.DataFrame(roc_list).transpose()
   temp_df.columns = df_columns
   roc_df = pd.concat([roc_df, temp_df], axis=0)

   temp_df = pd.DataFrame(prn_list).transpose()
   temp_df.columns = df_columns
   prn_df = pd.concat([prn_df, temp_df], axis=0)




... Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 0.378s
Cluster-based Local Outlier Factor ROC:0.7684, precision @ rank n:0.4643, execution time: 0.2452s
Feature Bagging ROC:0.7799, precision @ rank n:0.5, execution time: 1.381s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 0.1508s
Isolation Forest ROC:0.8527, precision @ rank n:0.5714, execution time: 0.9648s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.1958s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.1665s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: 1.3287s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: 0.1109s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5, execution time: 0.1235s

... Processing cardio.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.5763, precision @ rank n:0.1875, execution time: 1.0823s
Cluster-based Local Outlier Factor ROC:0.8221, precision @ rank n:0.4844, execution time: 0.4357s
Feature Bagging ROC:0.4879, precision @ rank n:0.1406, execution time: 1.7796s
Histogram-base Outlier Detection (HBOS) ROC:0.8453, precision @ rank n:0.4688, execution time: 0.0211s
Isolation Forest ROC:0.9414, precision @ rank n:0.5, execution time: 0.9194s
K Nearest Neighbors (KNN) ROC:0.6959, precision @ rank n:0.2812, execution time: 0.395s
Local Outlier Factor (LOF) ROC:0.4715, precision @ rank n:0.125, execution time: 0.2208s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-152.076751858648862 > -250.756822706501453). You may want to try with a higher value of support_fraction (current value: 0.511).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC:0.8778, precision @ rank n:0.3906, execution time: 1.1586s
One-class SVM (OCSVM) ROC:0.9507, precision @ rank n:0.5938, execution time: 0.1916s
Principal Component Analysis (PCA) ROC:0.9638, precision @ rank n:0.6875, execution time: 0.0283s

... Processing glass.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7104, precision @ rank n:0.25, execution time: 0.0959s
Cluster-based Local Outlier Factor ROC:0.8506, precision @ rank n:0.25, execution time: 0.1s
Feature Bagging ROC:0.7043, precision @ rank n:0.25, execution time: 0.0747s
Histogram-base Outlier Detection (HBOS) ROC:0.6524, precision @ rank n:0.0, execution time: 0.0045s
Isolation Forest ROC:0.7195, precision @ rank n:0.25, execution time: 0.7327s
K Nearest Neighbors (KNN) ROC:0.7805, precision @ rank n:0.25, execution time: 0.0206s
Local Outlier Factor (LOF) ROC:0.7774, precision @ rank n:0.25, execution time: 0.004s
Minimum Covariance Determinant (MCD) ROC:0.7165, precision @ rank n:

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.3486, precision @ rank n:0.0, execution time: 2.6671s
One-class SVM (OCSVM) ROC:0.4972, precision @ rank n:0.0, execution time: 3.6938s
Principal Component Analysis (PCA) ROC:0.504, precision @ rank n:0.0, execution time: 0.1252s

... Processing pendigits.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7008, precision @ rank n:0.0308, execution time: 4.1949s
Cluster-based Local Outlier Factor ROC:0.9609, precision @ rank n:0.3077, execution time: 0.6211s
Feature Bagging ROC:0.4687, precision @ rank n:0.0462, execution time: 13.0694s
Histogram-base Outlier Detection (HBOS) ROC:0.9294, precision @ rank n:0.2615, execution time: 0.0316s
Isolation Forest ROC:0.9422, precision @ rank n:0.2769, execution time: 2.1138s
K Nearest Neighbors (KNN) ROC:0.7602, precision @ rank n:0.0462, execution time: 1.4089s
Local Outlier Factor (LOF) ROC:0.481, precision @ rank n:0.0462, execution time: 1.4471s
Minimum Covariance Determinant (MCD) ROC:0.8271, precis

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.541490230701029 > -75.592253998160075). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.022992269210960 > -76.150022952275933). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.588090860494539 > -73.997089087121466). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\P

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.246740593074335 > -75.373826826202034). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.256978425484334 > -76.476983289741668). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.288199308055383 > -75.990589261845741). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\P

Minimum Covariance Determinant (MCD) ROC:0.9903, precision @ rank n:0.7534, execution time: 25.6294s
One-class SVM (OCSVM) ROC:0.9922, precision @ rank n:0.9553, execution time: 114.5001s
Principal Component Analysis (PCA) ROC:0.9902, precision @ rank n:0.9503, execution time: 0.1457s

... Processing vertebral.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.2797, precision @ rank n:0.0, execution time: 0.1073s
Cluster-based Local Outlier Factor ROC:0.3908, precision @ rank n:0.0, execution time: 0.1018s
Feature Bagging ROC:0.3027, precision @ rank n:0.0, execution time: 0.0797s
Histogram-base Outlier Detection (HBOS) ROC:0.2695, precision @ rank n:0.0, execution time: 0.005s
Isolation Forest ROC:0.3576, precision @ rank n:0.0, execution time: 0.7126s
K Nearest Neighbors (KNN) ROC:0.318, precision @ rank n:0.0, execution time: 0.0225s
Local Outlier Factor (LOF) ROC:0.318, precision @ rank n:0.0, execution time: 0.0051s
Minimum Covariance Determinant (MCD) ROC:0.3308, precision @ rank 

In [20]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,Iforest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5764,0.3824,0.4584,0.3486,0.4972,0.504
0,pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9422,0.7602,0.481,0.8271,0.93,0.9332


In [21]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,Iforest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2769,0.0462,0.0462,0.0615,0.2923,0.3385


In [22]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,Iforest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.378,0.2452,1.381,0.1508,0.9648,0.1958,0.1665,1.3287,0.1109,0.1235
0,cardio,1831,21,9.6122,1.0823,0.4357,1.7796,0.0211,0.9194,0.395,0.2208,1.1586,0.1916,0.0283
0,glass,214,9,4.2056,0.0959,0.1,0.0747,0.0045,0.7327,0.0206,0.004,0.0753,0,0.0036
0,ionosphere,351,33,35.8974,0.1624,0.1039,0.2035,0.0206,0.6866,0.0456,0.0168,0.3163,0.014,0.0313
0,letter,1600,32,6.25,0.8913,0.2446,1.5787,0.0243,0.925,0.3037,0.2369,2.1977,0.1538,0.0101
0,lympho,148,18,4.0541,0.0723,0.0884,0.0692,0.0106,0.6867,0.0127,0.0046,0.0745,0,0
0,mnist,7603,100,9.2069,17.8733,2.4884,129.401,0.1274,4.71,16.8508,16.4169,6.5619,10.7506,0.3874
0,musk,3062,166,3.1679,6.4,0.7917,34.9537,0.2091,3.6156,5.2682,4.6418,32.9718,2.678,0.2852
0,optdigits,5216,64,2.8758,6.8868,1.1462,38.3837,0.0933,2.2626,4.2545,3.9631,2.6671,3.6938,0.1252
0,pendigits,6870,16,2.2707,4.1949,0.6211,13.0694,0.0316,2.1138,1.4089,1.4471,4.313,2.2477,0.0171


# inferences  

#### aaryhtmia file has 14.6% outliers and PCA gives the least time duration for training and ABOD has 76% accuracy in predicting outliers.
#### letter file has 6.25% outliers and PCA gives the least time duration for training and Feature bagging has 86% accuracy in predicting outliers.
#### wbc file has 30% outliers and PCA gives the least time duration for training and HBOS has 70% accuracy in predicting outliers.
#### shuttle file has 9% outliers and HBOS gives the least time duration for training and HBOS has 98% accuracy in predicting outliers
#### vowels file has 12% outliers and PCA gives the least time duration for training and KNN has 86% accuracy in predicting outliers